In [ ]:

import os
import pickle

import pandas as pd
from sklearn.model_selection import train_test_split

import torch 
from torch_geometric.data import Data

# ------- H1: DATASET UTILS -------------

# ------- H2: Preprocess Data -------------
def window_scale_divison(df, W, T, company_to_id, ticker):
    """
        Returns the window of input and target values scaled by dividing with the
        last value of the previous window.

        Problems: With large W and T
    """
    SMOOTH = 0.00001
    list_df = [(
                    (df['Open'][i+1:i+W+1] / df['Open'][i:i+1].values+SMOOTH).values, 
                    (df['High'][i+1:i+W+1] / df['High'][i:i+1].values+SMOOTH).values,           
                    (df['Low'][i+1:i+W+1] / df['Low'][i:i+1].values+SMOOTH).values, 
                    (df['Close'][i+1:i+W+1] / df['Close'][i:i+1].values+SMOOTH).values,           
                    (df['Volume'][i+1:i+W+1] / (df['Volume'][i:i+1].values+SMOOTH)).values, 
                    company_to_id[ticker],  
                    df[i+1:i+W+1]['Date'], 
                    [
                        (df['Close'][i+W+T:i+W+T+1] / df['Close'][i+W:i+W+1].values).values
                        for T in [1, 5, 20]
                    ], 
                    df['Close'][i+W:i+W+1],
                    df.iloc[i+W:i+W+1, 7:].values,
                    [
                        (df['Low'][i+W+T:i+W+T+1]).values
                        for T in [0, 1, 5, 20]
                    ], 
                    [
                        (df['High'][i+W+T:i+W+T+1]).values
                        for T in [0, 1, 5, 20]
                    ]
                ) 
                for i in range(df.shape[0]-W-T)
            ]
    
    return list_df

In [22]:
directory = "data/" + "nasdaq_2" + "/"
W=20
T=5
company_to_id = {}
company_id    = 0

dataset = []
df_map = {}
skipped_ticker = []
total = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(filename)
        ticker, name = filename.split("-")
        df = pd.read_csv(f)
        df = df.fillna(method='ffill')

        if df.shape[0] <= 2800:    # 13 years
            print("Skipping file: Less Training-Testing samples [{0} samples]".format(df.shape[0]))
            skipped_ticker.append(ticker)
            continue
        total += 1

        if ticker not in company_to_id:
            company_to_id[ticker] = company_id
            company_id += 1
        
        
        if df.shape[0] > 2800:
            df = df.iloc[-2800:]
    
 
        list_df = [(
                df.iloc[i+W:i+W+1, 7:].shape
            ) 
            for i in range(df.shape[0]-W-T)
        ]

        list_df = window_scale_divison(df, W, T, company_to_id, ticker)
        df_map[company_to_id[ticker]] = list_df
        break

AMZN-Amazon.Com Inc..csv


In [38]:
type(list_df[0][0][0])
def normalize():
    

for i in [0,1,2,3]:
    list_df[0][0][i] = normalize(list_df[0][0][i])

numpy.ndarray

In [ ]:
save_path = "data/pickle/"+"nasdaqnew"+"/full_graph_data_correct-P25-W"+str(20)+"-T"+str(5)+"_"+str("True")+".pkl"


In [ ]:
save_dataset_graph(save_path, dataset, company_to_id, graph, hyper_data)

In [ ]:
if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument('--index', type=str, default="AAPL")
    #parser.add_argument('--window', type=int, default=10)
    #parser.add_argument('--test_size', type=float, default=0.2)

    #create_dataset("nasdaq100", 50, 5)
    d, s, c, g, h = create_batch_dataset("nasdaq100", 50, 5)
    print(len(d[0]), len(d[0][0]), d[0][0][0])